In [139]:
from google.colab import drive
drive.mount("/content/gdrive")
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import missingno as msno
from collections import Counter
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
###################
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from yellowbrick.model_selection import RFECV
from sklearn.feature_selection import RFE
import joblib
import re
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
!pip install autokeras
import autokeras as ak
from sklearn.metrics import confusion_matrix 
from sklearn.ensemble import RandomForestClassifier
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [140]:
df = pd.read_excel("/content/gdrive/MyDrive/Datos_Casos_Prueba.xlsx")
df.head(5)

,Tipo de Acero,X_min,X_max,Y_min,Y_max,Factor_Area,Perimetro_X,Perimentro_Y,Luminosidad_Suma,Luminosidad_min,...,Iso,Índice borde X,Índice borde Y,Índice Global externo,Ln(Índice_Areas),LN(Índice X),Ln(Índice Y),Índice de Orientación,Índice de Luminosidad,Área Sigmoide
0,403,543.0,548.0,1819724.0,1819728.0,16.0,5.0,4.0,2028.0,111.0,...,564.0,1.0,1.0,0.0,12041.0,699.0,6021.0,-2.0,-98.0,1322.0
1,440,482.0,490.0,2102085.0,2102099.0,68.0,14.0,14.0,7747.0,105.0,...,610.0,5714.0,1.0,1.0,18325.0,9031.0,11461.0,4286.0,-1099.0,2195.0
2,310,1088.0,1096.0,1094617.0,1094629.0,67.0,10.0,12.0,7416.0,94.0,...,190.0,8.0,1.0,1.0,18261.0,9031.0,10792.0,3333.0,-1353.0,2018.0
3,321,39.0,60.0,118073.0,118242.0,1590.0,143.0,172.0,154035.0,80.0,...,994.0,1469.0,9826.0,1.0,32014.0,13222.0,22279.0,8757.0,-2431.0,1.0
4,304,1613.0,1622.0,1752541.0,1752560.0,116.0,17.0,19.0,9105.0,65.0,...,963.0,5294.0,1.0,1.0,20645.0,9542.0,12787.0,5263.0,-3868.0,2942.0


In [141]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 29 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Tipo de Acero                  14 non-null     int64  
 1   X_min                          14 non-null     float64
 2   X_max                          14 non-null     float64
 3   Y_min                          14 non-null     float64
 4   Y_max                          14 non-null     float64
 5   Factor_Area                    14 non-null     float64
 6   Perimetro_X                    14 non-null     float64
 7            Perimentro_Y          14 non-null     float64
 8   Luminosidad_Suma               14 non-null     float64
 9   Luminosidad_min                14 non-null     float64
 10  Luminosidad_Max                14 non-null     float64
 11  Longitud Banda Transportadora  14 non-null     float64
 12  Día                            14 non-null     objec

In [142]:
df.drop("Iso", axis = 1, inplace = True)

In [143]:
df["Turno"].replace({'Día':'Dia','    Noche':'Noche',' ':''}, inplace = True)
df["Día"].replace({'Marte':'Martes','Miércoles':'Miercoles',}, inplace = True)

In [144]:
df["Día"].value_counts()

Martes       7
Miercoles    3
Lunes        2
Viernes      1
Jueves       1
Name: Día, dtype: int64

In [145]:
df["Turno"].value_counts()

Dia      10
Noche     4
Name: Turno, dtype: int64

In [146]:
df.rename(columns = {'         Perimentro_Y' :'Perimentro_Y','        Índice borde X':'Índice borde X','       Índice de Orientación':'Índice de Orientación'}, inplace = True)

In [147]:
df.describe(include = 'O')

,Día,Turno
count,14,14
unique,5,2
top,Martes,Dia
freq,7,10


In [148]:
df2 = pd.get_dummies(df, columns = ["Tipo de Acero","Día","Turno"])
df2.head(5)

,X_min,X_max,Y_min,Y_max,Factor_Area,Perimetro_X,Perimentro_Y,Luminosidad_Suma,Luminosidad_min,Luminosidad_Max,...,Tipo de Acero_420,Tipo de Acero_431,Tipo de Acero_440,Día_Jueves,Día_Lunes,Día_Martes,Día_Miercoles,Día_Viernes,Turno_Dia,Turno_Noche
0,543.0,548.0,1819724.0,1819728.0,16.0,5.0,4.0,2028.0,111.0,143.0,...,0,0,0,0,0,1,0,0,1,0
1,482.0,490.0,2102085.0,2102099.0,68.0,14.0,14.0,7747.0,105.0,127.0,...,0,0,1,0,0,0,0,1,1,0
2,1088.0,1096.0,1094617.0,1094629.0,67.0,10.0,12.0,7416.0,94.0,132.0,...,0,0,0,0,0,0,1,0,0,1
3,39.0,60.0,118073.0,118242.0,1590.0,143.0,172.0,154035.0,80.0,116.0,...,0,0,0,0,0,0,1,0,1,0
4,1613.0,1622.0,1752541.0,1752560.0,116.0,17.0,19.0,9105.0,65.0,94.0,...,0,0,0,0,0,1,0,0,0,1


In [149]:
df2.isna().any()

X_min                            False
X_max                            False
Y_min                            False
Y_max                            False
Factor_Area                      False
Perimetro_X                      False
Perimentro_Y                     False
Luminosidad_Suma                 False
Luminosidad_min                  False
Luminosidad_Max                  False
Longitud Banda Transportadora    False
Espesor                          False
Índice bordes                    False
Índice vacío                     False
Índice cuadrático                False
Índice externo X                 False
Índice borde X                   False
Índice borde Y                   False
Índice Global externo            False
Ln(Índice_Areas)                 False
LN(Índice X)                     False
Ln(Índice Y)                     False
Índice de Orientación            False
Índice de Luminosidad            False
Área Sigmoide                    False
Tipo de Acero_304        

In [150]:
Clases = ["Abolladuras","Deformaciones","Otras Fallas","Rayones Multiples",
                "Rayones Simples","Suciedad","Superficie Opaca"]
try:
  df2.insert(26, "Tipo de Acero_301", 0, allow_duplicates=False)
  df2.insert(27, "Tipo de Acero_302", 0, allow_duplicates=False)
  df2.insert(28, "Tipo de Acero_316", 0, allow_duplicates=False)
  df2.insert(29, "Tipo de Acero_416", 0, allow_duplicates=False)
except:
  pass
columnas_x = df2.columns
columnas_x
X = df2
scaler = StandardScaler()
df2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 43 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   X_min                          14 non-null     float64
 1   X_max                          14 non-null     float64
 2   Y_min                          14 non-null     float64
 3   Y_max                          14 non-null     float64
 4   Factor_Area                    14 non-null     float64
 5   Perimetro_X                    14 non-null     float64
 6   Perimentro_Y                   14 non-null     float64
 7   Luminosidad_Suma               14 non-null     float64
 8   Luminosidad_min                14 non-null     float64
 9   Luminosidad_Max                14 non-null     float64
 10  Longitud Banda Transportadora  14 non-null     float64
 11  Espesor                        14 non-null     float64
 12  Índice bordes                  14 non-null     float

In [151]:
X = np.concatenate((scaler.fit_transform(X[["X_min", "X_max","Y_min","Y_max","Factor_Area","Perimetro_X","Perimentro_Y","Luminosidad_Suma","Luminosidad_min","Luminosidad_Max",
                                                        "Longitud Banda Transportadora","Espesor","Índice bordes        ","Índice vacío","Índice cuadrático","Índice externo X","Índice borde X",
                                                        "Índice borde Y","Índice Global externo","Ln(Índice_Areas)","LN(Índice X)","Ln(Índice Y)","Índice de Orientación","Índice de Luminosidad",
                                                        "Área Sigmoide"]]),
                        X["Tipo de Acero_301"].to_numpy().reshape(len(X),1),
                        X["Tipo de Acero_302"].to_numpy().reshape(len(X),1),
                        X["Tipo de Acero_304"].to_numpy().reshape(len(X),1),
                        X["Tipo de Acero_310"].to_numpy().reshape(len(X),1),
                        X["Tipo de Acero_316"].to_numpy().reshape(len(X),1),
                        X["Tipo de Acero_321"].to_numpy().reshape(len(X),1),
                        X["Tipo de Acero_403"].to_numpy().reshape(len(X),1),
                        X["Tipo de Acero_416"].to_numpy().reshape(len(X),1),
                        X["Tipo de Acero_420"].to_numpy().reshape(len(X),1),
                        X["Tipo de Acero_431"].to_numpy().reshape(len(X),1),
                        X["Tipo de Acero_440"].to_numpy().reshape(len(X),1),
                        X["Día_Jueves"].to_numpy().reshape(len(X),1),
                        X["Día_Lunes"].to_numpy().reshape(len(X),1),
                        X["Día_Martes"].to_numpy().reshape(len(X),1),
                        X["Día_Miercoles"].to_numpy().reshape(len(X),1),
                        X["Día_Viernes"].to_numpy().reshape(len(X),1),
                        X["Turno_Dia"].to_numpy().reshape(len(X),1),
                        X["Turno_Noche"].to_numpy().reshape(len(X),1)), axis = 1)
pd.DataFrame(X, columns = columnas_x)

,X_min,X_max,Y_min,Y_max,Factor_Area,Perimetro_X,Perimentro_Y,Luminosidad_Suma,Luminosidad_min,Luminosidad_Max,...,Tipo de Acero_420,Tipo de Acero_431,Tipo de Acero_440,Día_Jueves,Día_Lunes,Día_Martes,Día_Miercoles,Día_Viernes,Turno_Dia,Turno_Noche
0,-0.058889,-0.100421,1.184541,1.184543,-0.417183,-0.447339,-0.544032,-0.428658,0.548558,0.142054,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,-0.181850,-0.220689,1.594429,1.594477,-0.407140,-0.401699,-0.481653,-0.419955,0.365705,-0.520865,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,1.039697,1.035907,0.131943,0.131875,-0.407333,-0.421984,-0.494129,-0.420459,0.030475,-0.313703,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,-1.074829,-1.112334,-1.285652,-1.285602,-0.113190,0.252466,0.503931,-0.197362,-0.396181,-0.976622,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,2.097968,2.126616,1.087015,1.087031,-0.397870,-0.386486,-0.450464,-0.417889,-0.853312,-1.888136,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5,-0.320937,-0.311927,-1.256046,-1.255637,1.327975,0.693647,2.125780,1.823275,-2.133280,2.462271,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
6,-0.905506,-0.963035,-0.317122,-0.317218,-0.405209,-0.396628,-0.462940,-0.414424,1.097115,0.722108,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
7,1.166690,1.172764,-0.199505,-0.199600,-0.403857,-0.401699,-0.500367,-0.416206,0.548558,-0.230838,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
8,0.088261,0.053025,1.772725,1.772795,-0.403471,-0.396628,-0.444226,-0.416229,0.487607,-0.520865,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
9,-0.953884,-0.998286,-1.160956,-1.161121,-0.399801,-0.345918,-0.469178,-0.412294,0.457131,-0.230838,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [152]:
model_logreg = joblib.load("/content/gdrive/MyDrive/Modelos_IF_MP3/reg_log.sav")
model_knn = joblib.load("/content/gdrive/MyDrive/Modelos_IF_MP3/clf_knn.sav")
model_rf = joblib.load("/content/gdrive/MyDrive/Modelos_IF_MP3/clf_rf.sav")
model_nn = tf.keras.models.load_model("/content/gdrive/MyDrive/Modelos_IF_MP3/model_autokeras/",
                                      custom_objects=ak.CUSTOM_OBJECTS)

In [153]:
Yp_log = model_logreg.predict(X)
Yp_knn = model_knn.predict(X)
Yp_rf = model_rf.predict(X)
predictions_nn = model_nn.predict(tf.expand_dims(X, -1))
Yp_nn = []
for i in range(len(predictions_nn)):
  Yp_nn.append(Clases[predictions_nn.argmax(axis=1)[i]])
Yp_nn

['Abolladuras',
 'Abolladuras',
 'Abolladuras',
 'Rayones Simples',
 'Superficie Opaca',
 'Otras Fallas',
 'Otras Fallas',
 'Otras Fallas',
 'Abolladuras',
 'Rayones Simples',
 'Rayones Multiples',
 'Abolladuras',
 'Otras Fallas',
 'Deformaciones']

In [155]:
Resultados = pd.DataFrame()
Resultados["Regresion logistica"] = Yp_log
Resultados["Clasificador KNN"] = Yp_knn
Resultados["Random Forest C"] = Yp_rf
Resultados["NeuralNetwork Autokeras"] = Yp_nn
Resultados.head(14)

,Regresion logistica,Clasificador KNN,Random Forest C,NeuralNetwork Autokeras
0,Deformaciones,Deformaciones,Deformaciones,Abolladuras
1,Abolladuras,Abolladuras,Deformaciones,Abolladuras
2,Abolladuras,Abolladuras,Deformaciones,Abolladuras
3,Rayones Simples,Rayones Simples,Otras Fallas,Rayones Simples
4,Abolladuras,Otras Fallas,Deformaciones,Superficie Opaca
5,Otras Fallas,Rayones Multiples,Otras Fallas,Otras Fallas
6,Otras Fallas,Abolladuras,Otras Fallas,Otras Fallas
7,Otras Fallas,Otras Fallas,Deformaciones,Otras Fallas
8,Otras Fallas,Abolladuras,Deformaciones,Abolladuras
9,Rayones Simples,Otras Fallas,Deformaciones,Rayones Simples


In [171]:
md = Resultados.mode(axis=1)
md[0]


0         Deformaciones
1           Abolladuras
2           Abolladuras
3       Rayones Simples
4           Abolladuras
5          Otras Fallas
6          Otras Fallas
7          Otras Fallas
8           Abolladuras
9       Rayones Simples
10    Rayones Multiples
11          Abolladuras
12         Otras Fallas
13        Deformaciones
Name: 0, dtype: object

In [173]:
Resultados["Desicion Final"] = md[0]
Resultados.head(14)

,Regresion logistica,Clasificador KNN,Random Forest C,NeuralNetwork Autokeras,Desicion Final
0,Deformaciones,Deformaciones,Deformaciones,Abolladuras,Deformaciones
1,Abolladuras,Abolladuras,Deformaciones,Abolladuras,Abolladuras
2,Abolladuras,Abolladuras,Deformaciones,Abolladuras,Abolladuras
3,Rayones Simples,Rayones Simples,Otras Fallas,Rayones Simples,Rayones Simples
4,Abolladuras,Otras Fallas,Deformaciones,Superficie Opaca,Abolladuras
5,Otras Fallas,Rayones Multiples,Otras Fallas,Otras Fallas,Otras Fallas
6,Otras Fallas,Abolladuras,Otras Fallas,Otras Fallas,Otras Fallas
7,Otras Fallas,Otras Fallas,Deformaciones,Otras Fallas,Otras Fallas
8,Otras Fallas,Abolladuras,Deformaciones,Abolladuras,Abolladuras
9,Rayones Simples,Otras Fallas,Deformaciones,Rayones Simples,Rayones Simples


In [174]:
ruta = "/content/gdrive/MyDrive/Modelos_IF_MP3/"
Resultados.to_csv(ruta+'results.csv')